In [6]:
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import json

In [15]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

tierminal_info = mt5.terminal_info()._asdict()
version= mt5.version()
print("MetaTrader5 version:", version)
df=pd.DataFrame(list(tierminal_info.items()),columns=['property','value'])
print(df)


MetaTrader5 version: (500, 4620, '11 Oct 2024')
                 property                                              value
0       community_account                                              False
1    community_connection                                              False
2               connected                                               True
3            dlls_allowed                                              False
4           trade_allowed                                               True
5       tradeapi_disabled                                              False
6           email_enabled                                              False
7             ftp_enabled                                              False
8   notifications_enabled                                              False
9                    mqid                                               True
10                  build                                               4620
11                maxbars   

In [16]:
account_info = mt5.account_info()._asdict()
df_account_info = pd.DataFrame(list(account_info.items()), columns=['property', 'value'])
print(df_account_info)

              property              value
0                login          312241304
1           trade_mode                  0
2             leverage               1000
3         limit_orders                300
4       margin_so_mode                  0
5        trade_allowed               True
6         trade_expert               True
7          margin_mode                  2
8      currency_digits                  2
9           fifo_close              False
10             balance           10022.72
11              credit                0.0
12              profit                0.0
13              equity           10022.72
14              margin                0.0
15         margin_free           10022.72
16        margin_level                0.0
17      margin_so_call               50.0
18        margin_so_so               20.0
19      margin_initial                0.0
20  margin_maintenance                0.0
21              assets                0.0
22         liabilities            

In [17]:
symbols=mt5.symbols_total()
print("Total symbols:",symbols)

Total symbols: 1505


In [19]:
symbols=mt5.symbols_get()

count=0
for s in symbols:
    count+=1
    print("{}. {}".format(count,s.name))
    if count==5: break
print()

1. EURUSD
2. GBPUSD
3. USDCHF
4. USDJPY
5. USDCAD



In [20]:
# get symbols containing RU in their names
ru_symbols=mt5.symbols_get("*RU*")
print('len(*RU*): ', len(ru_symbols))
for s in ru_symbols:
    print(s.name)
print()
 

len(*RU*):  2
EURUSD
NEARUSD



In [21]:
# get symbols whose names do not contain USD, EUR, JPY and GBP
group_symbols=mt5.symbols_get(group="*,!*USD*,!*EUR*,!*JPY*,!*GBP*")
print('len(*,!*USD*,!*EUR*,!*JPY*,!*GBP*):', len(group_symbols))
for s in group_symbols:
    print(s.name,":",s)

len(*,!*USD*,!*EUR*,!*JPY*,!*GBP*): 1396
AUDNZD : SymbolInfo(custom=False, chart_mode=0, select=False, visible=False, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=0, digits=5, spread=0, spread_float=True, ticks_bookdepth=0, trade_calc_mode=0, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=0, trade_freeze_level=0, trade_exemode=2, swap_mode=1, swap_rollover3days=3, margin_hedged_use_leg=False, expiration_mode=15, filling_mode=2, order_mode=127, order_gtc_mode=0, option_mode=0, option_right=0, bid=0.0, bidhigh=0.0, bidlow=0.0, ask=0.0, askhigh=0.0, asklow=0.0, last=0.0, lasthigh=0.0, lastlow=0.0, volume_real=0.0, volumehigh_real=0.0, volumelow_real=0.0, option_strike=0.0, point=1e-05, trade_tick_value=0.5834700000000002, trade_tick_value_profit=0.5834700000000002, trade_tick_value_loss=0.58375, trade_tick_size=1e-05, trade_contract_size=100000.0, trade_accrued_interest=0.0, trade_face_value=0.0, trade_liquidity_

In [25]:
symbol_info=mt5.symbol_info("EURJPY")
if symbol_info!=None:
    # display the terminal data 'as is'    
    df=pd.DataFrame(list(symbol_info._asdict().items()),columns=['property','value'])
    print(df)

         property                         value
0          custom                         False
1      chart_mode                             0
2          select                         False
3         visible                         False
4   session_deals                             0
..            ...                           ...
91        formula                              
92           isin                              
93           name                        EURJPY
94           page                              
95           path  Forex\Standard\Majors\EURJPY

[96 rows x 2 columns]


In [34]:
selected=mt5.symbol_select("GBPUSD",True)
print("GBPUSD selected=",selected)

GBPUSD selected= True


In [33]:
lasttick=mt5.symbol_info_tick("GBPUSD")
df=pd.DataFrame(lasttick._asdict(),index=[0])
df['time']=pd.to_datetime(df['time'],unit='s')
print(df)


                 time      bid      ask  last  volume       time_msc  flags  \
0 2024-11-26 10:21:19  1.25494  1.25515   0.0       0  1732616479534   1124   

   volume_real  
0          0.0  
